In [1]:
import tribolium_clustering as tc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

prediction_folder = 'C:/Users/ryans/OneDrive/Documents/Master Thesis/Documents/Figures/Detecting Developmental Domains Lund PCA//'
lund_folder = 'D:/Uni/MSTER TUD/Master Thesis/output data/Lund Gastrulation Correct voxel sizes//'

lund = tc.processed_dataset(lund_folder)

In [2]:
def dataset_to_unit_props(dataset):
    combined_thesis_props = dataset.get_combined_thesis_props_no_correlation()
    
    # conversion factors for measurements with units
    volume_cf = 0.69*0.69*0.69
    distance_cf = 0.69

    conversion_factors = [volume_cf,volume_cf,1,distance_cf,1,
                     1,1,1,1,
                     1,1,1,
                     distance_cf,
                     distance_cf,1,
                     1]

    regionprops_w_units = combined_thesis_props * conversion_factors
    
    return regionprops_w_units

def props_predictions_to_cluster_dataframes(regionprops,predictions):
    predictions_df = pd.DataFrame({'prediction':predictions})
    regprops_w_predictions = pd.concat([regionprops,predictions_df],axis = 1)

    cluster_regprops_list = [regprops_w_predictions[regprops_w_predictions['prediction']==i] for i in range(5)]
    
    return cluster_regprops_list

predictions = np.load(prediction_folder+'predictions PCA 7 components, Kmeans++ 2 to 8 clusters.npy')[3]


unit_props = dataset_to_unit_props(lund)
prediction_df_lund = props_predictions_to_cluster_dataframes(unit_props,predictions)

No Predictions in Regionprops of D:/Uni/MSTER TUD/Master Thesis/output data/Lund Gastrulation Correct voxel sizes//


In [3]:
plot_cmap = tc.plot_cmap()
names = ['Cluster {}'.format(i+1) for i in range(5)]


def plot_cluster_comparison(measurements,x_axis_title,names,
                            kwargs = dict(hist_kws={'alpha':0.3}, kde_kws={'linewidth':0.5})):
    
    # Renaming X-Axis
    new_name_props = [prop.rename(x_axis_title) for prop in measurements]
    
    
    kwargs = dict(hist_kws={'alpha':0.3}, kde_kws={'linewidth':1})
    plt.figure(figsize =(4,3), dpi = 400)
    
    for i,Area, name in zip(range(5),new_name_props, names):
        sns.distplot(Area, label = name,color=plot_cmap[i] ,**kwargs)
    plt.legend()
    


def plot_cluster_comparison_tnc(measurements,x_axis_title,names,
                            kwargs = dict(hist_kws={'alpha':0.3}, kde_kws={'linewidth':1}),
                            bins = None):
    
    # Renaming X-Axis
    new_name_props = [prop.rename(x_axis_title) for prop in measurements]
    
    
    
    plt.figure(figsize =(4,3), dpi = 400)
    
    for i,Area, name in zip(range(5),new_name_props, names):
        sns.distplot(Area, bins=bins ,label = name,color=plot_cmap[i] ,**kwargs)
    plt.legend()
    

In [6]:
def make_plots_of_clusters(cluster_props,cluster_names,output_folder):
    cluster_regprops_list =cluster_props

    areas = [prop['area'] for prop in cluster_regprops_list]

    names = cluster_names

    
    plot_cluster_comparison(areas,'Volume [\u03BCm\u00b3]',names)
    plt.savefig(output_folder+'area.png', dpi =  400, format = 'png', bbox_inches = 'tight')
    plt.close()
    
    bbox_area = [prop['bbox_area'] for prop in cluster_regprops_list]
    plot_cluster_comparison(bbox_area, 'Bounding Box Volume [\u03BCm\u00b3]',names)
    plt.savefig(output_folder+'bbox area.png', dpi =  400, format = 'png', bbox_inches = 'tight')
    plt.close()
    
    extent = [prop['extent'] for prop in cluster_regprops_list]
    plot_cluster_comparison(extent, 'Extent',names)
    plt.savefig(output_folder+'extent.png', dpi =  400, format = 'png', bbox_inches = 'tight')
    plt.close()
    
    feret_diam = [prop['feret_diameter_max'] for prop in cluster_regprops_list]
    plot_cluster_comparison(feret_diam, 'Maximum Feret-Diameter [\u03BCm]',names)
    plt.xlim([10,55])
    plt.savefig(output_folder+'feret diameter.png', dpi =  400, format = 'png', bbox_inches = 'tight')
    plt.close()
    
    max_int = [prop['max_intensity'] for prop in cluster_regprops_list]
    plot_cluster_comparison(max_int, 'Maximum Intensity',names)
    plt.savefig(output_folder+'max intensity.png', dpi =  400, format = 'png', bbox_inches = 'tight')
    plt.close()
    
    min_int = [prop['min_intensity'] for prop in cluster_regprops_list]
    plot_cluster_comparison(min_int, 'Minimum Intensity',names)
    plt.savefig(output_folder+'min intensity.png', dpi =  400, format = 'png', bbox_inches = 'tight')
    plt.close()
    
    mean_int = [prop['mean_intensity'] for prop in cluster_regprops_list]
    plot_cluster_comparison(mean_int, 'Mean Intensity',names)
    plt.savefig(output_folder+'mean intensity.png', dpi =  400, format = 'png', bbox_inches = 'tight')
    plt.close()
    
    stdev_int = [prop['image_stdev'] for prop in cluster_regprops_list]
    plot_cluster_comparison(stdev_int, 'Standard Deviation of Intensity',names)
    plt.savefig(output_folder+'stdev intensity.png', dpi =  400, format = 'png', bbox_inches = 'tight')
    plt.close()
    
    distance_cf = 0.69
    cent_0 = [prop['centroid-0']*distance_cf for prop in cluster_regprops_list]
    plot_cluster_comparison(cent_0, 'Location Z [\u03BCm]',names)
    plt.savefig(output_folder+'Location Z.png', dpi =  400, format = 'png', bbox_inches = 'tight')
    plt.close()
    
    cent_1 = [prop['centroid-1']*distance_cf for prop in cluster_regprops_list]
    plot_cluster_comparison(cent_1, 'Location Y [\u03BCm]',names)
    plt.savefig(output_folder+'Location Y.png', dpi =  400, format = 'png', bbox_inches = 'tight')
    plt.close()
    
    cent_2 = [prop['centroid-2']*distance_cf for prop in cluster_regprops_list]
    plot_cluster_comparison(cent_2, 'Location X [\u03BCm]',names)
    plt.xlim([0,550])
    plt.savefig(output_folder+'Location X.png', dpi =  400, format = 'png', bbox_inches = 'tight')
    plt.close()
    
    avg_dist = [prop['avg distance of 6 closest points'] for prop in cluster_regprops_list]
    plot_cluster_comparison(avg_dist, 'Avg. Distance to 6 Neighbours [\u03BCm]',names)
    plt.xlim([5,45])
    plt.savefig(output_folder+'Avg. Distance to 6 Neighbours.png', dpi =  400, format = 'png', bbox_inches = 'tight')
    plt.close()
    
    stdev_dist = [prop['stddev distance of 6 closest points'] for prop in cluster_regprops_list]
    plot_cluster_comparison(stdev_dist, 'Std. Dev. to 6 Neighbours [\u03BCm]',names)
    plt.savefig(output_folder+'stdev Distance to 6 Neighbours.png', dpi =  400, format = 'png', bbox_inches = 'tight')
    plt.close()
    
    tnc = [prop['touching neighbor count'] for prop in cluster_regprops_list]
    plot_cluster_comparison_tnc(tnc, 'Touching Neighbour Count',names, bins = range(31), 
                        kwargs = dict(hist_kws={'alpha':0.3}, 
                                      kde_kws={'linewidth':1, 
                                              'bw':0.2}))
    plt.savefig(output_folder+'touching neighbour count.png', dpi =  400, format = 'png', bbox_inches = 'tight')
    plt.close()
    
    aspect = [prop['aspect_ratio'] for prop in cluster_regprops_list]
    plot_cluster_comparison(aspect, 'Aspect Ratio',names)
    plt.xlim([0.5,3])
    plt.savefig(output_folder+'aspect ratio.png', dpi =  400, format = 'png', bbox_inches = 'tight')
    
    plt.close()
    
    sol = [prop['solidity'] for prop in cluster_regprops_list]

    plot_cluster_comparison(sol, 'Solidity',names)
    plt.xlim([0.2,1])
    plt.savefig(output_folder+'solidity.png', dpi =  400, format = 'png', bbox_inches = 'tight')
    plt.close()


In [7]:
lund_plot_folder = tc.get_folder_path('lund plots')
make_plots_of_clusters(prediction_df_lund,names,lund_plot_folder)

C:\Users\ryans\anaconda3\envs\mstr_v4\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
C:\Users\ryans\anaconda3\envs\mstr_v4\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
C:\Users\ryans\anaconda3\envs\mstr_v4\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar 

C:\Users\ryans\anaconda3\envs\mstr_v4\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
C:\Users\ryans\anaconda3\envs\mstr_v4\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
C:\Users\ryans\anaconda3\envs\mstr_v4\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar 

C:\Users\ryans\anaconda3\envs\mstr_v4\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
C:\Users\ryans\anaconda3\envs\mstr_v4\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
C:\Users\ryans\anaconda3\envs\mstr_v4\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar 

C:\Users\ryans\anaconda3\envs\mstr_v4\lib\site-packages\seaborn\distributions.py:1657: FutureWarning: The `bw` parameter is deprecated in favor of `bw_method` and `bw_adjust`. Using 0.2 for `bw_method`, but please see the docs for the new parameters and update your code.
  warnings.warn(msg, FutureWarning)
C:\Users\ryans\anaconda3\envs\mstr_v4\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
C:\Users\ryans\anaconda3\envs\mstr_v4\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histo